# Functions for predicting results from trained models

In [7]:
import pickle
import numpy as np

class MyLinear:
    """Class that holds our custom linear regression model"""
    
    def __init__(self, thetas):
        self.thetas = thetas

    def predict(self, X, thresh=None):
        this_x = X[0][:]
        this_x = [1, *this_x] # Add bias
        this_x = np.array(this_x)

        n_labels = self.thetas.shape[1] - 1
        preds = np.zeros((n_labels))

        for i in range(n_labels):
            pred = np.matmul(np.transpose(this_x), self.thetas[i])
            preds[i] = self.pred_round(pred, thresh) if thresh else pred

        return np.array([preds])

    def pred_round(self, x, thresh):
        return 0 if x < thresh else 1

def predict_cross_val(trained_models, examples, is_regressor=True, num_labels=9):
    """Takes trained models and returns predictions on new data"""
    
    num_splits = len(trained_models)
    num_examples = examples.shape[0]
    
    # init prediction array
    if num_labels > 1:
        preds = np.zeros((num_examples, num_labels))
    else:
        preds = np.zeros(num_examples)

    # get predictions on test data
    for i in range(num_examples):
        # init predictions for this specific example
        if num_labels > 1:
            pred = np.zeros((num_splits, num_labels))
        else:
            pred = np.zeros(num_splits)
        
        # get prediction from all models for this
        # specific example
        for j in range(num_splits):
            if num_labels > 1:
                pred[j,:] = trained_models[j].predict([examples[i]])
            else:
                pred[j] = trained_models[j].predict([examples[i]])
        
        # Average prediction across all models
        # and round if this is a classifier model
        if num_labels > 1:
            pred = np.mean(pred, 0)
            pred = pred if is_regressor else np.around(pred)
        else:
            pred = np.mean(pred)
            pred = pred if is_regressor else np.round(pred)
        
        # Save prediction
        if num_labels > 1:
            preds[i,:] = pred
        else:
            preds[i] = pred
    
    return preds

## Examples

In [3]:
# linear_model = pickle.load(open('linear.sav','rb'))
# predict_cross_val(linear_model, np.array([[1, -1, 0, 0, 0, 0, 0, 1, 0]]))

In [4]:
# knn_model = pickle.load(open('knn.sav','rb'))
# predict_cross_val(knn_model, np.array([[1, -1, 0, 0, 0, 0, 0, 1, 0]]))

In [5]:
# mlp_model = pickle.load(open('mlp.sav','rb'))
# predict_cross_val(mlp_model, np.array([[1, -1, 0, 0, 0, 0, 0, 1, 0]]))